In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 22 13:36:21 2020

@author: shiro
"""
import numpy as np
import pandas as pd
import re
import string

# Задание 1

Разработать систему предсказания следующего слова по предыдущим.

Воспользоваться данными с сайта http://norvig.com/ngrams/ или найти другие словари частот встречаемости слов.

In [2]:
def text_prepare(text):
    """
        text: a string

        return: modified string
    """
    # Перевести символы в нижний регистр
    text = text.lower() #your code

    # Заменить символы пунктуации на пробелы
    text = re.sub(r'[{}]'.format(string.punctuation), ' ', text)

    # Удалить "плохие" символы
    text = re.sub('[^A-Za-z0-9 ]', '', text)
    return text

def preprocess(file_name: str, amount_of_columns: int = 2, text_preparation = False):
    dict_columns = {2:['k', 'nu'],
                    3:['k-1', 'k', 'nu']}
    with open(file_name, 'r') as file:
        lines = file.readlines()
    count_words = np.array(list(map(lambda x: x.split(), lines)))
    k_words = pd.DataFrame(data=count_words, columns=dict_columns[amount_of_columns])
    k_words['nu'] = k_words['nu'].astype(dtype=np.int64)
    if text_preparation:
        k_words['k'] = k_words['k'].apply(text_prepare)
        if amount_of_columns == 3:
            k_words['k-1'] = k_words['k-1'].apply(text_prepare)
    return k_words

def final_preprocess(text_preparation = False):
    onegrams = preprocess(file_name='count_1w.txt', amount_of_columns=2, text_preparation=text_preparation)
    bigrams = preprocess(file_name='count_2w.txt', amount_of_columns=3, text_preparation=text_preparation)
    bigrams = bigrams.groupby(by = ['k-1','k'], as_index=False).sum()
    onegrams[r'P(w_k)'] = onegrams['nu'] / onegrams['nu'].sum()
    onegrams[r'log P(w_k)'] = np.log(onegrams[r'P(w_k)'])
    bigrams['log P(w_k-1|w_k)'] = np.log(bigrams['nu'] / bigrams['nu'].sum())
    return onegrams, bigrams

def predict_next_word(onegrams,bigrams_full, word):
   candidates = bigrams_full[bigrams_full['k-1']==word]
   logs = onegrams[onegrams['k'].isin(candidates['k'])].iloc[:,3] + candidates.iloc[:,3].values
   try:
       result = onegrams.iloc[logs.index[np.argmax(logs)]]['k']
       return result
   except:
       return f"Word {word} is not in vocalbulary"


## Without Text Preparation

In [3]:
onegrams, bigrams = final_preprocess(False)

In [4]:
words = ['will', 'love', 'continue', 'establish', 'divide', 'the', 'world', 'such', 'famous', 'same']

next_word = [(word, predict_next_word(onegrams, bigrams, word = word)) for word in words]
no_preparation_result  = pd.DataFrame(data = next_word, columns = ['word', 'next_word'])
no_preparation_result

,word,next_word
0,will,each
1,love,in
2,continue,shopping
3,establish,the
4,divide,the
5,the,it
6,world,a
7,such,the
8,famous,for
9,same,as


## With Text Preparation

In [5]:
onegrams, bigrams = final_preprocess(True)

In [6]:
next_word_with_preparation = [(word, predict_next_word(onegrams, bigrams, word = word)) for word in words]
next_word_with_preparation_result  = pd.DataFrame(data = next_word_with_preparation, columns = ['word', 'next_word'])
next_word_with_preparation_result

,word,next_word
0,will,each
1,love,in
2,continue,a
3,establish,the
4,divide,the
5,the,it
6,world,a
7,such,the
8,famous,for
9,same,as


## Задание 2

Использована 6-ая модель, учитывающая два слова перед словом.

In [7]:
bigrams['k-2'] = np.where(bigrams['k-1'].isin(bigrams['k']),bigrams['k'],1)
bigrams['nu-2'] = np.where(bigrams['k-1'].isin(bigrams['k']),bigrams['nu'],1)

In [8]:
bigrams['log P(w_k-2|w_k)'] = np.log(bigrams['nu-2'] / bigrams['nu-2'].sum())

In [9]:
def predict_next_word_m(onegrams,bigrams_full, word):
   candidates = bigrams_full[bigrams_full['k-1']==word]
   logs = onegrams[onegrams['k'].isin(candidates['k'])].iloc[:,3] + candidates.iloc[:,3].values + candidates.iloc[:, 6]
   try:
       result = onegrams.iloc[logs.index[np.argmax(logs)]]['k']
       return result
   except:
       return f"Word {word} is not in vocalbulary"

In [10]:
next_word_with_preparation = [(word, predict_next_word_m(onegrams, bigrams, word = word)) for word in words]
next_word_with_preparation_result  = pd.DataFrame(data = next_word_with_preparation, columns = ['word', 'next_word'])
next_word_with_preparation_result

,word,next_word
0,will,hvorfor
1,love,bocode
2,continue,admires
3,establish,neoliberalism
4,divide,deviously
5,the,sesh
6,world,backchannel
7,such,camerakodak
8,famous,chugach
9,same,mentat
